# MNISTでの機械学習ハンズオン

機械学習について

機械学習の種類としては大雑把に
+ 深層学習 (Deep Learning) 
+ 強化学習
+ ベイズ推定

があげられます

+ 画像処理
    + 物体検知
    + 物体認識 (分類とか)
    + 画像生成 https://blogs.nvidia.com/blog/2019/03/18/gaugan-photorealistic-landscapes-nvidia-research/
+ 自然言語処理
    + 翻訳
    + 文章理解
    + 概要生成
    + 文章生成
+ 音声認識
    + 文章に変換
+ 数値データによる予測 
    + 株の数値予測
    + 競馬の予測
    + 顧客予測

それぞれでトレンドとされる技術が違うので勉強が必要

だいたいGoogle APIが提供してたりする。
+ 分かち書きや重要単語, 係り受けなどを返すAPI
https://cloud.google.com/natural-language/ 

+ みんな大好き Google 翻訳
https://cloud.google.com/translate/

+ 深層学習 (物体検知 https://www.youtube.com/watch?v=tJnGiPGz-p8) 
+ 強化学習 (自動運転 https://www.youtube.com/watch?time_continue=27&v=SZYbEhLrvg4)
+ ベイズ推定 (ベイズ推定でできること https://www.udemy.com/computervision/learn/lecture/297992#overview)

## Kerasで多層パーセプトロン（Multi-Layer Perceptron, MLP）を組んで手書き文字を判別してみよう

※このパートはPyDataOkinawa 第16回の資料を参考にしています

Neural Networkと呼ばれる技術で、

色々派生した技術がありますが、大本の技術から触っていこうかなと思います。

機械学習をするなら、機械学習の知識についてよく知ってる上で、

フレームワークがどのような動きをするかどうかを知ることが大事です。

## モデルの構築 

In [ ]:
## 必要なフレームワークのインストール
# !pip install -y keras
# !conda install -y tensorflow

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop

In [ ]:
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(784,)))
model.add(Dense(512, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [ ]:
# model = Sequential()
# model.add(Dense(512, activation='relu', input_shape=(784,)))
# model.add(Dropout(0.2))
# model.add(Dense(512, activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(10, activation='softmax'))


## モデルの可視化

In [ ]:
model.summary()

## モデルのコンパイル
+ 最適化手法の選択
    + 通常はAdadeltaかAdam、もしくはSGD with Nesterov momentumでOK
    + RNNならRMSpropが安定している
    + 超複雑なネットワークを、大量のデータで学習させるときには極力小さな学習率でシンプルなSGD without momentumを用いるのが良かったりする。（validation errorの減少が止まったら学習率を半分にする）

In [ ]:
# ==== デフォルト設定で使う場合 ====
# optimizerに全て小文字のストリングを渡すだけ（超カンタン！）
# 例えば以下のような設定が使える
# optimizer='sgd'
# optimizer='adadelta'
# optimizer='adam'
# optimizer='rmsprop

# ==== 好みに合わせて設定を変えたい場合 ====
# まず、最適化手法のクラスを読み込む
# from keras.optimizers import SGD, Adadelta, Adam, RMSprop
# それらをインスタンス化してからoptimizer引数に渡す
# 例）　SGD with Nesterov momentum を用いる場合
# sgd_nesterov=SGD(lr=0.01, momentum=0.9, nesterov=True)

# モデルのコンパイル
model.compile(loss='categorical_crossentropy', # 損失関数（この量のパラメータ勾配で学習する）
              optimizer='sgd', # 最適化手法（デフォルト設定）
              #optimizer='rmsprop', # 最適化手法（デフォルト設定）
              #optimizer=sgd_nesterov, # 最適化手法（お好み設定）
              metrics=['accuracy'] # 評価指標
             )

## モデルの保存

In [ ]:
# JSON形式でモデルを保存
json_string = model.to_json()
open('./mnist_mlp_model.json', 'w').write(json_string)

In [ ]:
# YAML形式でモデルを保存
yaml_string = model.to_yaml()
open('./mnist_mlp_model.yaml', 'w').write(yaml_string)

## パラメータの保存

In [ ]:
#　初期ウエイトの保存
model.save_weights('./mnist_mlp_init_weight.hdf5', overwrite=True)

## モデルの読み出し

In [ ]:
from keras.models import model_from_json, model_from_yaml

In [ ]:
# JSON形式で保存されたモデルの呼び出し
#json_string = open('./mnist_mlp_model.json', 'r').read()
#model = model_from_json(json_string)

In [ ]:
# YAML形式で保存されたモデルの呼び出し
#yaml_string = open('./mnist_mlp_model.yaml', 'r').read()
#model = model_from_yaml(yaml_string)

## パラメータの読み出し

In [ ]:
#model.load_weights('./mnist_mlp_init_weight.hdf5')


## MNISTデータセットの読み込み

In [ ]:
from keras.datasets import mnist

In [ ]:
# 手書き文字データセット（MNIST）の読み込み
(X_train, y_train), (X_test, y_test) = mnist.load_data()

## データの可視化

In [ ]:
# 可視化用ライブラリの読み込み
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
# 入力データを可視化（最初の５文字）
fig, ax = plt.subplots(1, 5)

for ii in range(5):
    ax[ii].imshow(X_train[ii], cmap='gray')
    ax[ii].axis('off')

## データの前処理

In [ ]:
X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)
X_train = X_train.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')


In [ ]:
from keras.utils import np_utils

In [ ]:
nb_classes = 10

In [ ]:
# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

In [ ]:
Y_train[:5]

In [ ]:
y_train[:5]

## 学習

In [ ]:
# # TensorBoardで学習の進捗状況をみる (今回は見ない)
# tb_cb = keras.callbacks.TensorBoard(log_dir='/tmp/keras_mnist_mlp', histogram_freq=1)

# バリデーションロスが下がれば、エポックごとにモデルを保存
cp_cb = keras.callbacks.ModelCheckpoint(filepath='./mnist_mlp_best_weight.hdf5', 
                                        monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

# バリデーションロスが５エポック連続で上がったら、ランを打ち切る
es_cb = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto')

cbks = [cp_cb, es_cb]

In [ ]:
import time

In [ ]:
tic = time.time()

# 学習を実行
# 学習途中の損失関数の値などはhistory.historyに保存される。
history = model.fit(X_train, Y_train,   # 画像とラベルデータ
                    batch_size=128, 
                    nb_epoch=20,    # エポック数の指定
                    verbose=1,     # ログ出力の指定. 0だとログが出ない
                    validation_data=(X_test, Y_test),
                    callbacks=cbks)
toc = time.time()

# 学習にかかった時間を表示
print("Execution time: {0:.2f} [sec]".format(toc - tic))

## ベストなパラメータの呼び出し

In [ ]:
model.load_weights('./mnist_mlp_best_weight.hdf5')

## 学習結果の評価

In [ ]:
# テストデータに対する評価値
score = model.evaluate(X_test, Y_test, verbose=0)

In [ ]:
print('Test score:', score[0]) # 損失関数の値
print('Test accuracy:', score[1]) # 精度

In [ ]:
# 学習曲線
fig, ax = plt.subplots(1, 2, figsize=(12, 6))
ax[0].set_title('Training performance (Loss)')
ax[0].plot(history.epoch, history.history['loss'], label='loss')
ax[0].plot(history.epoch, history.history['val_loss'], label='val_loss')
ax[0].set(xlabel='Epoch', ylabel='Loss')
ax[0].legend()

ax[1].set_title('Training performance (Accuracy)')
ax[1].plot(history.epoch, history.history['acc'], label='acc')
ax[1].plot(history.epoch, history.history['val_acc'], label='val_acc')
ax[1].set(xlabel='Epoch', ylabel='Accuracy')
ax[1].legend(loc='best')

In [ ]:
# 予測値
Y_test_pred = model.predict(X_test)

In [ ]:
# 予測の形
Y_test_pred.shape

In [ ]:
# 予測の可視化
plt.imshow(Y_test_pred[:10], cmap='gray', interpolation='nearest', vmin=0, vmax=1)

In [ ]:
# 入力データを可視化（最初の10文字）
fig, ax = plt.subplots(1, 10, figsize=(10, 2))

for ii in range(10):
    ax[ii].imshow(X_test[ii].reshape(28, 28), cmap='gray')
    ax[ii].axis('off')

In [ ]:
plt.imshow(Y_test_pred[:40], cmap='gray', interpolation='nearest', vmin=0, vmax=1)

In [ ]:
# 予測の可視化
plt.imshow(Y_test_pred[30:40], cmap='gray', interpolation='nearest', vmin=0, vmax=1)

In [ ]:
# 入力データを可視化（30~39文字目までの10文字）
fig, ax = plt.subplots(1, 10, figsize=(10, 2))

for ii in range(10):
    ax[ii].imshow(X_test[ii+30].reshape(28, 28), cmap='gray')
    ax[ii].axis('off')

## もし機械学習に興味を持ったら

+ PyData.Okinawa の過去資料を漁る (機械学習に関連する知識と基盤を固める)
+ https://python-beginners-okinawa.connpass.com/event/127905/ ベイズの勉強会に参加する
+ データ分析の競技プログラミングに参加する
    + Kaggle
    + SIGNATE
+ 最近のトレンドを追う
    + SSD + HoloV3 ( 物体検知 )
    + GAN ( 画像生成 )
    + BERT, Self-Attention ( 自然言語処理 )
    + CNN, RNN ( 深層学習 )
    + PyTorch あたりで触れることが出来る